In [1]:
import os
import datetime
import time

import IPython
import IPython.display
from ipywidgets import IntProgress
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

from sklearn.metrics import r2_score
import math

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

C:\Users\iasha\AppData\Local\Temp\ipykernel_2552\424092037.py:11: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def convert_line_to_list(line: str):
    split_line = line.split(' ')
    result_list = []

    for current in split_line:
        if current == '\n':
            continue
            
        result_list.append(int(current))

    return result_list

In [4]:
def get_mlt(clustering_unit_index):
    return clustering_unit_index // 80 * 0.25

def get_magneticLat(clustering_unit_index):
    return clustering_unit_index % 80 * 0.5 + 50

In [5]:
def cluster_unit_index_to_file_path(clustering_unit_index):
    mlt = get_mlt(clustering_unit_index)
    magnetic_lat = get_magneticLat(clustering_unit_index)
    
    return f"C:\\Coding\\final-qualifying-work_2023-2024\\OvationPrimeDataCollector\\north-nowcast-diffuse\\{mlt:.2f}_{magnetic_lat:.1f}.csv"

In [6]:
def get_cluster_centre(cluster):
    cluster_centre = pd.read_csv(cluster_unit_index_to_file_path(cluster[0]), sep=';', low_memory=False)

    f = IntProgress(min=0, max=len(cluster))
    display(f)
    i = 0
    for clustering_unit_index in cluster:
        # if i % 100 == 0:
        #     f.value += 100
        f.value += 1
        if i == 0:
            i += 1
            continue
        current_df = pd.read_csv(cluster_unit_index_to_file_path(clustering_unit_index), sep=';', low_memory=False)
        cluster_centre['y'] += current_df['y']
        i += 1

    cluster_centre['y'] = cluster_centre['y'].divide(len(cluster))
    return cluster_centre

In [7]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df, val_df, test_df,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [8]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [9]:
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.utils.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

In [10]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [11]:
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')
    
  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [34]:
# output_file = open("computational-experiments-output.txt", "w")

In [12]:
def investigate(cluster_index, model, conv_width, clusters, save_data):
    cluster_centre = get_cluster_centre(clusters[cluster_index])
    cluster_centre = cluster_centre.drop(['date'], axis=1)
    cluster_centre = cluster_centre.drop(['segment'], axis=1)
    
    n = len(cluster_centre)
    train_df = cluster_centre[0:int(n*0.7)]
    val_df = cluster_centre[int(n*0.7):int(n*0.9)]
    test_df = cluster_centre[int(n*0.9):]

    train_mean = train_df.mean()
    train_std = train_df.std()

    train_df = (train_df - train_mean) / train_std
    val_df = (val_df - train_mean) / train_std
    test_df = (test_df - train_mean) / train_std
    
    conv_window = WindowGenerator(
    input_width=conv_width,
    label_width=1,
    shift=1,
    train_df=train_df,
    val_df=val_df,
    test_df=test_df,
    label_columns=['y']
    )

    model = keras.models.Sequential.from_config(model.get_config())
    
    history = compile_and_fit(model, conv_window)

    inputs, labels = conv_window.example

    val_df_numpy = val_df.to_numpy()
    train_df_numpy = train_df.to_numpy()
    new_true_values_val = []
    new_true_values_train = []
    new_input_val = []
    new_input_train = []

    new_true_values_val = val_df_numpy[3:].flatten()
    new_input_val = np.column_stack([val_df_numpy[0:-3], val_df_numpy[1:-2], val_df_numpy[2:-1]])

    new_true_values_train = train_df_numpy[3:].flatten()
    new_input_train = np.column_stack([train_df_numpy[0:-3], train_df_numpy[1:-2], train_df_numpy[2:-1]])
        
    val_predictions = model(np.array(new_input_val)).numpy()
    train_predictions = model(np.array(new_input_train)).numpy()
    
    new_val_predictions = val_predictions.flatten()
    new_train_predictions = train_predictions.flatten()
    
    new_true_values_val = (new_true_values_val * train_std['y']) + train_mean['y']
    new_val_predictions = (new_val_predictions * train_std['y']) + train_mean['y']

    new_true_values_train = (new_true_values_train * train_std['y']) + train_mean['y']
    new_train_predictions = (new_train_predictions * train_std['y']) + train_mean['y']

    print("***TRAIN***:")
    output_file.write("***TRAIN***:\n")
    print(f"correlation related to cluster centre: {np.corrcoef(new_true_values_train, new_train_predictions)[0][1]:.2f}")
    output_file.write(f"correlation related to cluster centre: {np.corrcoef(new_true_values_train, new_train_predictions)[0][1]:.2f}\n")
    print(f"r2 score related to cluster centre: {r2_score(new_true_values_train, new_train_predictions):.2f}")
    output_file.write(f"r2 score related to cluster centre: {r2_score(new_true_values_train, new_train_predictions):.2f}\n")
    print("***VALIDATION***:")
    output_file.write("***VALIDATION***:\n")
    print(f"correlation related to cluster centre: {np.corrcoef(new_true_values_val, new_val_predictions)[0][1]:.2f}")
    output_file.write(f"correlation related to cluster centre: {np.corrcoef(new_true_values_val, new_val_predictions)[0][1]:.2f}\n")
    print(f"r2 score related to cluster centre: {r2_score(new_true_values_val, new_val_predictions):.2f}")
    output_file.write(f"r2 score related to cluster centre: {r2_score(new_true_values_val, new_val_predictions):.2f}\n")

    sum_correlation_val = 0
    sum_r2_val = 0

    f = IntProgress(min=0, max=len(clusters[cluster_index]))
    display(f)

    i = 0
    for clustering_unit_index in clusters[cluster_index]:
        if (i % 100 == 0):
            f.value += 100
        
        current_df = pd.read_csv(cluster_unit_index_to_file_path(clustering_unit_index), sep=';', low_memory=False)
        current_df = current_df.drop(['date'], axis=1)
        current_df = current_df.drop(['segment'], axis=1)

        n = len(current_df)
        current_train_df = current_df[0:int(n*0.7)]
        current_val_df = current_df[int(n*0.7):int(n*0.9)]
        current_test_df = current_df[int(n*0.9):]

        current_train_df = (current_train_df - train_mean) / train_std
        current_val_df = (current_val_df - train_mean) / train_std
        current_test_df = (current_test_df - train_mean) / train_std

        current_val_df_numpy = current_val_df.to_numpy()
        current_new_true_values_val = current_val_df_numpy[3:].flatten()
        current_new_input_val = np.column_stack([current_val_df_numpy[0:-3], current_val_df_numpy[1:-2], current_val_df_numpy[2:-1]])

        current_val_predictions = model(np.array(current_new_input_val)).numpy()
        current_new_val_predictions = current_val_predictions.flatten()

        current_new_true_values_val = (current_new_true_values_val * train_std['y']) + train_mean['y']
        current_new_val_predictions = (current_new_val_predictions * train_std['y']) + train_mean['y']
            
        sum_correlation_val += np.corrcoef(current_new_true_values_val, current_new_val_predictions)[0][1]
        sum_r2_val += r2_score(current_new_true_values_val, current_new_val_predictions)

        i += 1

    if (save_data):
        f = open(f"train mean and std for cluster {cluster_index}.txt", "w")
        f.write(str(train_mean.mean()) + ' ' + str(train_std.mean()))
        f.close()
        model.save(f"model for cluster {cluster_index}")

    cluster_len = len(clusters[cluster_index])
    
    return sum_correlation_val / cluster_len, sum_r2_val / cluster_len,

In [13]:
def do_investigations(model, clusters, conv_width):
    cluster_sizes = []
    correlations = []
    r2_scores = []

    for i in range(len(clusters)):
        print(f"---do_investigations. cluster index : {i}---")
        output_file.write(f"---do_investigations. cluster index : {i}---\n")
        average_correlation, average_r2 = investigate(i, model, conv_width, clusters, False)
        
        print("***VALIDATION***:")
        output_file.write("***VALIDATION***:\n")
        print(f"average correlation: {average_correlation:.2f}")
        output_file.write(f"average correlation: {average_correlation:.2f}\n")
        print(f"average r2: {average_r2:.2f}")
        output_file.write(f"average r2: {average_r2:.2f}\n")

        cluster_sizes.append(len(clusters[i]))
        correlations.append(average_correlation)
        r2_scores.append(average_r2)

    return cluster_sizes, correlations, r2_scores

In [14]:
def print_metrics(cluster_sizes, correlations, r2_scores, clusters):
    total_size = 0
    correlation_sum = 0
    r2_score_sum = 0
    
    bad_clusters_indices = []

    for i in range(len(cluster_sizes)):
        if (math.isnan(correlations[i]) or (r2_scores[i] < 0)):
            bad_clusters_indices.append(i)
            continue
        
        correlation_sum += cluster_sizes[i] * correlations[i]
        r2_score_sum += cluster_sizes[i] * r2_scores[i]
        total_size += cluster_sizes[i]

    weighted_average_correlation = correlation_sum / total_size
    weighted_average_r2_score = r2_score_sum / total_size

    print(f"weighted_average_correlation: {weighted_average_correlation:.2f}")
    output_file.write(f"weighted_average_correlation: {weighted_average_correlation:.2f}\n")
    print(f"weighted_average_r2_score: {weighted_average_r2_score:.2f}")
    output_file.write(f"weighted_average_r2_score: {weighted_average_r2_score:.2f}\n")

    bad_clusters_size_sum = 0
    total_clusters_size_sum = 0

    for cluster_size in cluster_sizes:
        total_clusters_size_sum += cluster_size
    
    for bad_cluster_index in bad_clusters_indices:
        bad_clusters_size_sum += len(clusters[bad_cluster_index])

    bad_clusters_rate = bad_clusters_size_sum / total_clusters_size_sum

    print(f"bad clusters rate: {bad_clusters_rate:.2f}%")
    output_file.write(f"bad clusters rate: {bad_clusters_rate:.2f}%\n")

In [15]:
def show_results(model, clusters, conv_width):
    cluster_sizes, correlations, r2_scores = do_investigations(model, clusters, conv_width)
    print_metrics(cluster_sizes, correlations, r2_scores, clusters)

In [16]:
CONV_WIDTH = 3

In [17]:
multi_step_dense_model = tf.keras.Sequential([
    # Shape: (time, features) => (time*features)
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=1),
    # Add back the time dimension.
    # Shape: (outputs) => (1, outputs)
    tf.keras.layers.Reshape([1, -1]),
])

In [18]:
lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.Conv1D(filters=32,
                           kernel_size=(CONV_WIDTH,),
                           activation='relu'),
    tf.keras.layers.LSTM(128, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1)
])

In [19]:
rnn_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.Conv1D(filters=32,
                           kernel_size=(CONV_WIDTH,),
                           activation='relu'),
    tf.keras.layers.SimpleRNN(128, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1)
])

In [20]:
def get_clusters(path):
    file = open(path, 'r')
    lines = file.readlines()
    file.close()

    clusters = []

    for line in lines:
        if line.strip() == '':
            continue
        clusters.append(convert_line_to_list(line))

    return clusters

In [22]:
first_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1.txt"
second_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 3 0.1 0.1.txt"
third_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 2 0.1 0.1.txt"
fourth_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 1 0.1 0.1.txt"

output_file = open("computational-experiments-output", "a")

selected_path = first_path

print(f"msd {os.path.split(selected_path)[-1]}")
output_file.write(f"msd {os.path.split(selected_path)[-1]}\n")
show_results(multi_step_dense_model, get_clusters(selected_path), CONV_WIDTH)

output_file.close()

msd kohonen 15 0.1 0.1.txt
---do_investigations. cluster index : 0---


IntProgress(value=0, max=5788)

Epoch 1/20

368/368 [==============================] - 2s 3ms/step - loss: 0.2709 - mean_absolute_error: 0.3076 - val_loss: 0.2332 - val_mean_absolute_error: 0.2651
Epoch 2/20
368/368 [==============================] - 1s 2ms/step - loss: 0.1963 - mean_absolute_error: 0.2694 - val_loss: 0.2304 - val_mean_absolute_error: 0.2685
Epoch 3/20
368/368 [==============================] - 1s 2ms/step - loss: 0.1949 - mean_absolute_error: 0.2687 - val_loss: 0.2286 - val_mean_absolute_error: 0.2612
Epoch 4/20
368/368 [==============================] - 1s 2ms/step - loss: 0.1931 - mean_absolute_error: 0.2683 - val_loss: 0.2281 - val_mean_absolute_error: 0.2620
Epoch 5/20
368/368 [==============================] - 1s 2ms/step - loss: 0.1930 - mean_absolute_error: 0.2672 - val_loss: 0.2257 - val_mean_absolute_error: 0.2679
Epoch 6/20
368/368 [==============================] - 1s 2ms/step - loss: 0.1926 - mean_absolute_error: 0.2679 - val_loss: 0.2255 - val_mean_absolute_error: 0.2660
Epoch 7/20
368/

IntProgress(value=0, max=5788)

***VALIDATION***:
average correlation: nan
average r2: -6211285110130805857920745472.00
---do_investigations. cluster index : 1---


...

***VALIDATION***:
average correlation: 0.91
average r2: 0.83
---do_investigations. cluster index : 10---


IntProgress(value=0, max=1)

Epoch 1/20
368/368 [==============================] - 2s 3ms/step - loss: 0.2573 - mean_absolute_error: 0.3460 - val_loss: 0.1823 - val_mean_absolute_error: 0.2971
Epoch 2/20
368/368 [==============================] - 1s 3ms/step - loss: 0.1674 - mean_absolute_error: 0.2815 - val_loss: 0.1792 - val_mean_absolute_error: 0.2982
Epoch 3/20
368/368 [==============================] - 1s 3ms/step - loss: 0.1660 - mean_absolute_error: 0.2800 - val_loss: 0.1799 - val_mean_absolute_error: 0.2915
Epoch 4/20
368/368 [==============================] - 1s 3ms/step - loss: 0.1658 - mean_absolute_error: 0.2795 - val_loss: 0.1796 - val_mean_absolute_error: 0.2925
***TRAIN***:
correlation related to cluster centre: 0.91
r2 score related to cluster centre: 0.84
***VALIDATION***:
correlation related to cluster centre: 0.91
r2 score related to cluster centre: 0.82


IntProgress(value=0, max=1)

***VALIDATION***:
average correlation: 0.91
average r2: 0.82
weighted_average_correlation: 0.91
weighted_average_r2_score: 0.74
bad clusters rate: 0.75%


In [23]:
first_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1.txt"
second_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 3 0.1 0.1.txt"
third_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 2 0.1 0.1.txt"
fourth_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 1 0.1 0.1.txt"

output_file = open("computational-experiments-output", "a")

selected_path = second_path

print(f"msd {os.path.split(selected_path)[-1]}")
output_file.write(f"msd {os.path.split(selected_path)[-1]}\n")
show_results(multi_step_dense_model, get_clusters(selected_path), CONV_WIDTH)

output_file.close()

msd kohonen 15 0.1 0.1 - 3 0.1 0.1.txt
---do_investigations. cluster index : 0---


IntProgress(value=0, max=654)

Epoch 1/20
368/368 [==============================] - 2s 3ms/step - loss: 0.2329 - mean_absolute_error: 0.3438 - val_loss: 0.1940 - val_mean_absolute_error: 0.3129
Epoch 2/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2025 - mean_absolute_error: 0.3152 - val_loss: 0.2020 - val_mean_absolute_error: 0.3186
Epoch 3/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2030 - mean_absolute_error: 0.3158 - val_loss: 0.1921 - val_mean_absolute_error: 0.3100
Epoch 4/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2023 - mean_absolute_error: 0.3153 - val_loss: 0.2030 - val_mean_absolute_error: 0.3255
Epoch 5/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2015 - mean_absolute_error: 0.3151 - val_loss: 0.1943 - val_mean_absolute_error: 0.3176
***TRAIN***:
correlation related to cluster centre: 0.90
r2 score related to cluster centre: 0.80
***VALIDATION***:
correlation related to cluster centre: 0.91
r2 score related to 

IntProgress(value=0, max=654)

***VALIDATION***:
average correlation: 0.91
average r2: 0.66
---do_investigations. cluster index : 1---


...

***VALIDATION***:
average correlation: 0.90
average r2: 0.80
---do_investigations. cluster index : 15---


IntProgress(value=0, max=5)

Epoch 1/20
368/368 [==============================] - 2s 3ms/step - loss: 0.3024 - mean_absolute_error: 0.2407 - val_loss: 0.3043 - val_mean_absolute_error: 0.2716
Epoch 2/20
368/368 [==============================] - 1s 2ms/step - loss: 0.1924 - mean_absolute_error: 0.2005 - val_loss: 0.3247 - val_mean_absolute_error: 0.2693
Epoch 3/20
368/368 [==============================] - 1s 2ms/step - loss: 0.1903 - mean_absolute_error: 0.1992 - val_loss: 0.3312 - val_mean_absolute_error: 0.2747
***TRAIN***:
correlation related to cluster centre: 0.90
r2 score related to cluster centre: 0.81
***VALIDATION***:
correlation related to cluster centre: 0.91
r2 score related to cluster centre: 0.82


IntProgress(value=0, max=5)

***VALIDATION***:
average correlation: 0.91
average r2: 0.81
weighted_average_correlation: 0.90
weighted_average_r2_score: 0.77
bad clusters rate: 0.59%


In [24]:
first_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1.txt"
second_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 3 0.1 0.1.txt"
third_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 2 0.1 0.1.txt"
fourth_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 1 0.1 0.1.txt"

output_file = open("computational-experiments-output", "a")

selected_path = third_path

print(f"msd {os.path.split(selected_path)[-1]}")
output_file.write(f"msd {os.path.split(selected_path)[-1]}\n")
show_results(multi_step_dense_model, get_clusters(selected_path), CONV_WIDTH)

output_file.close()

msd kohonen 15 0.1 0.1 - 2 0.1 0.1.txt
---do_investigations. cluster index : 0---


IntProgress(value=0, max=222)

Epoch 1/20
368/368 [==============================] - 2s 3ms/step - loss: 0.2792 - mean_absolute_error: 0.3435 - val_loss: 0.1777 - val_mean_absolute_error: 0.2809
Epoch 2/20
368/368 [==============================] - 1s 2ms/step - loss: 0.2081 - mean_absolute_error: 0.2987 - val_loss: 0.1760 - val_mean_absolute_error: 0.2863
Epoch 3/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2080 - mean_absolute_error: 0.2969 - val_loss: 0.1746 - val_mean_absolute_error: 0.2777
Epoch 4/20
368/368 [==============================] - 1s 2ms/step - loss: 0.2073 - mean_absolute_error: 0.2966 - val_loss: 0.1750 - val_mean_absolute_error: 0.2736
Epoch 5/20
368/368 [==============================] - 1s 2ms/step - loss: 0.2070 - mean_absolute_error: 0.2957 - val_loss: 0.1739 - val_mean_absolute_error: 0.2812
Epoch 6/20
368/368 [==============================] - 1s 2ms/step - loss: 0.2056 - mean_absolute_error: 0.2946 - val_loss: 0.1740 - val_mean_absolute_error: 0.2820
Epoch 7/20
368/3

IntProgress(value=0, max=222)

***VALIDATION***:
average correlation: 0.90
average r2: 0.80
---do_investigations. cluster index : 1---


...

***VALIDATION***:
average correlation: 0.90
average r2: 0.81
---do_investigations. cluster index : 20---


IntProgress(value=0, max=164)

Epoch 1/20
368/368 [==============================] - 2s 3ms/step - loss: 0.2208 - mean_absolute_error: 0.3157 - val_loss: 0.1420 - val_mean_absolute_error: 0.2378
Epoch 2/20
368/368 [==============================] - 1s 3ms/step - loss: 0.1794 - mean_absolute_error: 0.2707 - val_loss: 0.1406 - val_mean_absolute_error: 0.2424
Epoch 3/20
368/368 [==============================] - 1s 3ms/step - loss: 0.1769 - mean_absolute_error: 0.2678 - val_loss: 0.1385 - val_mean_absolute_error: 0.2312
Epoch 4/20
368/368 [==============================] - 1s 3ms/step - loss: 0.1780 - mean_absolute_error: 0.2680 - val_loss: 0.1392 - val_mean_absolute_error: 0.2340
Epoch 5/20
368/368 [==============================] - 1s 2ms/step - loss: 0.1778 - mean_absolute_error: 0.2670 - val_loss: 0.1398 - val_mean_absolute_error: 0.2419
***TRAIN***:
correlation related to cluster centre: 0.91
r2 score related to cluster centre: 0.83
***VALIDATION***:
correlation related to cluster centre: 0.92
r2 score related to 

IntProgress(value=0, max=164)

***VALIDATION***:
average correlation: 0.90
average r2: 0.77
weighted_average_correlation: 0.90
weighted_average_r2_score: 0.77
bad clusters rate: 0.53%


In [25]:
first_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1.txt"
second_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 3 0.1 0.1.txt"
third_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 2 0.1 0.1.txt"
fourth_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 1 0.1 0.1.txt"

output_file = open("computational-experiments-output", "a")

selected_path = fourth_path

print(f"msd {os.path.split(selected_path)[-1]}")
output_file.write(f"msd {os.path.split(selected_path)[-1]}\n")
show_results(multi_step_dense_model, get_clusters(selected_path), CONV_WIDTH)

output_file.close()

msd kohonen 15 0.1 0.1 - 1 0.1 0.1.txt
---do_investigations. cluster index : 0---


IntProgress(value=0, max=269)

Epoch 1/20
368/368 [==============================] - 2s 3ms/step - loss: 0.2703 - mean_absolute_error: 0.3634 - val_loss: 0.1672 - val_mean_absolute_error: 0.2868
Epoch 2/20
368/368 [==============================] - 1s 2ms/step - loss: 0.1897 - mean_absolute_error: 0.3005 - val_loss: 0.1663 - val_mean_absolute_error: 0.2889
Epoch 3/20
368/368 [==============================] - 1s 2ms/step - loss: 0.1876 - mean_absolute_error: 0.2981 - val_loss: 0.1653 - val_mean_absolute_error: 0.2855
Epoch 4/20
368/368 [==============================] - 1s 2ms/step - loss: 0.1879 - mean_absolute_error: 0.2983 - val_loss: 0.1617 - val_mean_absolute_error: 0.2774
Epoch 5/20
368/368 [==============================] - 1s 2ms/step - loss: 0.1871 - mean_absolute_error: 0.2977 - val_loss: 0.1653 - val_mean_absolute_error: 0.2821
Epoch 6/20
368/368 [==============================] - 1s 3ms/step - loss: 0.1869 - mean_absolute_error: 0.2972 - val_loss: 0.1619 - val_mean_absolute_error: 0.2798
***TRAIN***:
cor

IntProgress(value=0, max=269)

***VALIDATION***:
average correlation: 0.91
average r2: 0.76
---do_investigations. cluster index : 1---


...

***VALIDATION***:
average correlation: 0.90
average r2: 0.80
---do_investigations. cluster index : 44---


IntProgress(value=0, max=1)

Epoch 1/20
368/368 [==============================] - 2s 3ms/step - loss: 0.2420 - mean_absolute_error: 0.2072 - val_loss: 0.3374 - val_mean_absolute_error: 0.2329
Epoch 2/20
368/368 [==============================] - 1s 2ms/step - loss: 0.1877 - mean_absolute_error: 0.1789 - val_loss: 0.3412 - val_mean_absolute_error: 0.2280
Epoch 3/20
368/368 [==============================] - 1s 3ms/step - loss: 0.1855 - mean_absolute_error: 0.1770 - val_loss: 0.3374 - val_mean_absolute_error: 0.2260
***TRAIN***:
correlation related to cluster centre: 0.91
r2 score related to cluster centre: 0.82
***VALIDATION***:
correlation related to cluster centre: 0.90
r2 score related to cluster centre: 0.81


IntProgress(value=0, max=1)

***VALIDATION***:
average correlation: 0.90
average r2: 0.81
weighted_average_correlation: 0.90
weighted_average_r2_score: 0.77
bad clusters rate: 0.49%


In [26]:
first_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1.txt"
second_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 3 0.1 0.1.txt"
third_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 2 0.1 0.1.txt"
fourth_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 1 0.1 0.1.txt"

output_file = open("computational-experiments-output", "a")

selected_path = first_path

print(f"lstm {os.path.split(selected_path)[-1]}")
output_file.write(f"lstm {os.path.split(selected_path)[-1]}\n")
show_results(lstm_model, get_clusters(selected_path), CONV_WIDTH)

output_file.close()

lstm kohonen 15 0.1 0.1.txt
---do_investigations. cluster index : 0---


IntProgress(value=0, max=5788)

Epoch 1/20
368/368 [==============================] - 6s 6ms/step - loss: 0.2555 - mean_absolute_error: 0.3158 - val_loss: 0.2562 - val_mean_absolute_error: 0.2779
Epoch 2/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1976 - mean_absolute_error: 0.2714 - val_loss: 0.2391 - val_mean_absolute_error: 0.2642
Epoch 3/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1983 - mean_absolute_error: 0.2715 - val_loss: 0.2398 - val_mean_absolute_error: 0.2671
Epoch 4/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1961 - mean_absolute_error: 0.2698 - val_loss: 0.2375 - val_mean_absolute_error: 0.2674
Epoch 5/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1967 - mean_absolute_error: 0.2704 - val_loss: 0.2645 - val_mean_absolute_error: 0.2828
Epoch 6/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1958 - mean_absolute_error: 0.2702 - val_loss: 0.2476 - val_mean_absolute_error: 0.2729
***TRAIN***:
cor

IntProgress(value=0, max=5788)

***VALIDATION***:
average correlation: nan
average r2: -978905577997787022305001472.00
---do_investigations. cluster index : 1---


...

***VALIDATION***:
average correlation: 0.91
average r2: 0.83
---do_investigations. cluster index : 10---


IntProgress(value=0, max=1)

Epoch 1/20
368/368 [==============================] - 4s 6ms/step - loss: 0.2235 - mean_absolute_error: 0.3273 - val_loss: 0.1840 - val_mean_absolute_error: 0.3140
Epoch 2/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1665 - mean_absolute_error: 0.2799 - val_loss: 0.1787 - val_mean_absolute_error: 0.2995
Epoch 3/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1665 - mean_absolute_error: 0.2796 - val_loss: 0.1798 - val_mean_absolute_error: 0.3044
Epoch 4/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1666 - mean_absolute_error: 0.2808 - val_loss: 0.1797 - val_mean_absolute_error: 0.3035
***TRAIN***:
correlation related to cluster centre: 0.91
r2 score related to cluster centre: 0.84
***VALIDATION***:
correlation related to cluster centre: 0.91
r2 score related to cluster centre: 0.82


IntProgress(value=0, max=1)

***VALIDATION***:
average correlation: 0.91
average r2: 0.82
weighted_average_correlation: 0.90
weighted_average_r2_score: 0.75
bad clusters rate: 0.75%


In [27]:
first_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1.txt"
second_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 3 0.1 0.1.txt"
third_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 2 0.1 0.1.txt"
fourth_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 1 0.1 0.1.txt"

output_file = open("computational-experiments-output", "a")

selected_path = second_path

print(f"lstm {os.path.split(selected_path)[-1]}")
output_file.write(f"lstm {os.path.split(selected_path)[-1]}\n")
show_results(lstm_model, get_clusters(selected_path), CONV_WIDTH)

output_file.close()

lstm kohonen 15 0.1 0.1 - 3 0.1 0.1.txt
---do_investigations. cluster index : 0---


IntProgress(value=0, max=654)

Epoch 1/20
368/368 [==============================] - 4s 6ms/step - loss: 0.2590 - mean_absolute_error: 0.3617 - val_loss: 0.1938 - val_mean_absolute_error: 0.3026
Epoch 2/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2039 - mean_absolute_error: 0.3175 - val_loss: 0.1927 - val_mean_absolute_error: 0.2988
Epoch 3/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2043 - mean_absolute_error: 0.3180 - val_loss: 0.1919 - val_mean_absolute_error: 0.2985
Epoch 4/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2020 - mean_absolute_error: 0.3155 - val_loss: 0.1909 - val_mean_absolute_error: 0.2986
Epoch 5/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2016 - mean_absolute_error: 0.3144 - val_loss: 0.1910 - val_mean_absolute_error: 0.2999
Epoch 6/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2017 - mean_absolute_error: 0.3147 - val_loss: 0.1920 - val_mean_absolute_error: 0.3002
***TRAIN***:
cor

IntProgress(value=0, max=654)

***VALIDATION***:
average correlation: 0.90
average r2: 0.78
---do_investigations. cluster index : 1---


...

***VALIDATION***:
average correlation: 0.90
average r2: 0.79
---do_investigations. cluster index : 15---


IntProgress(value=0, max=5)

Epoch 1/20
368/368 [==============================] - 5s 6ms/step - loss: 0.2921 - mean_absolute_error: 0.2495 - val_loss: 0.3290 - val_mean_absolute_error: 0.2832
Epoch 2/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1938 - mean_absolute_error: 0.2007 - val_loss: 0.3167 - val_mean_absolute_error: 0.2768
Epoch 3/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1919 - mean_absolute_error: 0.1991 - val_loss: 0.3138 - val_mean_absolute_error: 0.2703
Epoch 4/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1895 - mean_absolute_error: 0.1990 - val_loss: 0.3141 - val_mean_absolute_error: 0.2603
Epoch 5/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1897 - mean_absolute_error: 0.1974 - val_loss: 0.3133 - val_mean_absolute_error: 0.2641
Epoch 6/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1887 - mean_absolute_error: 0.1972 - val_loss: 0.3146 - val_mean_absolute_error: 0.2713
Epoch 7/20
368/3

IntProgress(value=0, max=5)

***VALIDATION***:
average correlation: 0.91
average r2: 0.82
weighted_average_correlation: 0.90
weighted_average_r2_score: 0.77
bad clusters rate: 0.59%


In [28]:
first_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1.txt"
second_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 3 0.1 0.1.txt"
third_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 2 0.1 0.1.txt"
fourth_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 1 0.1 0.1.txt"

output_file = open("computational-experiments-output", "a")

selected_path = third_path

print(f"lstm {os.path.split(selected_path)[-1]}")
output_file.write(f"lstm {os.path.split(selected_path)[-1]}\n")
show_results(lstm_model, get_clusters(selected_path), CONV_WIDTH)

output_file.close()

lstm kohonen 15 0.1 0.1 - 2 0.1 0.1.txt
---do_investigations. cluster index : 0---


IntProgress(value=0, max=222)

Epoch 1/20
368/368 [==============================] - 5s 6ms/step - loss: 0.2709 - mean_absolute_error: 0.3452 - val_loss: 0.1757 - val_mean_absolute_error: 0.2798
Epoch 2/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2083 - mean_absolute_error: 0.2985 - val_loss: 0.1743 - val_mean_absolute_error: 0.2786
Epoch 3/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2078 - mean_absolute_error: 0.2977 - val_loss: 0.1753 - val_mean_absolute_error: 0.2770
Epoch 4/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2085 - mean_absolute_error: 0.2983 - val_loss: 0.1738 - val_mean_absolute_error: 0.2801
Epoch 5/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2063 - mean_absolute_error: 0.2959 - val_loss: 0.1750 - val_mean_absolute_error: 0.2740
Epoch 6/20
368/368 [==============================] - 2s 5ms/step - loss: 0.2079 - mean_absolute_error: 0.2975 - val_loss: 0.1742 - val_mean_absolute_error: 0.2844
***TRAIN***:
cor

IntProgress(value=0, max=222)

***VALIDATION***:
average correlation: 0.90
average r2: 0.81
---do_investigations. cluster index : 1---


...

***VALIDATION***:
average correlation: 0.89
average r2: 0.80
---do_investigations. cluster index : 20---


IntProgress(value=0, max=164)

Epoch 1/20
368/368 [==============================] - 5s 6ms/step - loss: 0.2514 - mean_absolute_error: 0.3227 - val_loss: 0.1484 - val_mean_absolute_error: 0.2463
Epoch 2/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1774 - mean_absolute_error: 0.2690 - val_loss: 0.1397 - val_mean_absolute_error: 0.2354
Epoch 3/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1773 - mean_absolute_error: 0.2678 - val_loss: 0.1454 - val_mean_absolute_error: 0.2454
Epoch 4/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1764 - mean_absolute_error: 0.2676 - val_loss: 0.1434 - val_mean_absolute_error: 0.2402
***TRAIN***:
correlation related to cluster centre: 0.91
r2 score related to cluster centre: 0.82
***VALIDATION***:
correlation related to cluster centre: 0.92
r2 score related to cluster centre: 0.84


IntProgress(value=0, max=164)

***VALIDATION***:
average correlation: 0.90
average r2: 0.72
weighted_average_correlation: 0.90
weighted_average_r2_score: 0.76
bad clusters rate: 0.53%


In [29]:
first_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1.txt"
second_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 3 0.1 0.1.txt"
third_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 2 0.1 0.1.txt"
fourth_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 1 0.1 0.1.txt"

output_file = open("computational-experiments-output", "a")

selected_path = fourth_path

print(f"lstm {os.path.split(selected_path)[-1]}")
output_file.write(f"lstm {os.path.split(selected_path)[-1]}\n")
show_results(lstm_model, get_clusters(selected_path), CONV_WIDTH)

output_file.close()

lstm kohonen 15 0.1 0.1 - 1 0.1 0.1.txt
---do_investigations. cluster index : 0---


IntProgress(value=0, max=269)

Epoch 1/20
368/368 [==============================] - 5s 6ms/step - loss: 0.2600 - mean_absolute_error: 0.3522 - val_loss: 0.1656 - val_mean_absolute_error: 0.2846
Epoch 2/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1891 - mean_absolute_error: 0.3000 - val_loss: 0.1640 - val_mean_absolute_error: 0.2821
Epoch 3/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1891 - mean_absolute_error: 0.2997 - val_loss: 0.1647 - val_mean_absolute_error: 0.2788
Epoch 4/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1876 - mean_absolute_error: 0.2983 - val_loss: 0.1628 - val_mean_absolute_error: 0.2717
Epoch 5/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1892 - mean_absolute_error: 0.2997 - val_loss: 0.1613 - val_mean_absolute_error: 0.2736
Epoch 6/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1880 - mean_absolute_error: 0.2987 - val_loss: 0.1620 - val_mean_absolute_error: 0.2826
Epoch 7/20
368/3

IntProgress(value=0, max=269)

***VALIDATION***:
average correlation: 0.90
average r2: 0.76
---do_investigations. cluster index : 1---


...

***VALIDATION***:
average correlation: 0.90
average r2: 0.80
---do_investigations. cluster index : 44---


IntProgress(value=0, max=1)

Epoch 1/20
368/368 [==============================] - 6s 6ms/step - loss: 0.3045 - mean_absolute_error: 0.2257 - val_loss: 0.3540 - val_mean_absolute_error: 0.2271
Epoch 2/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1918 - mean_absolute_error: 0.1810 - val_loss: 0.3475 - val_mean_absolute_error: 0.2363
Epoch 3/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1910 - mean_absolute_error: 0.1805 - val_loss: 0.3437 - val_mean_absolute_error: 0.2298
Epoch 4/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1887 - mean_absolute_error: 0.1798 - val_loss: 0.3432 - val_mean_absolute_error: 0.2274
Epoch 5/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1864 - mean_absolute_error: 0.1798 - val_loss: 0.3408 - val_mean_absolute_error: 0.2249
Epoch 6/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1873 - mean_absolute_error: 0.1792 - val_loss: 0.3496 - val_mean_absolute_error: 0.2321
Epoch 7/20
368/3

IntProgress(value=0, max=1)

***VALIDATION***:
average correlation: 0.90
average r2: 0.81
weighted_average_correlation: 0.89
weighted_average_r2_score: 0.77
bad clusters rate: 0.49%


In [30]:
first_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1.txt"
second_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 3 0.1 0.1.txt"
third_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 2 0.1 0.1.txt"
fourth_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 1 0.1 0.1.txt"

output_file = open("computational-experiments-output", "a")

selected_path = first_path

print(f"rnn {os.path.split(selected_path)[-1]}")
output_file.write(f"rnn {os.path.split(selected_path)[-1]}\n")
show_results(rnn_model, get_clusters(selected_path), CONV_WIDTH)

output_file.close()

rnn kohonen 15 0.1 0.1.txt
---do_investigations. cluster index : 0---


IntProgress(value=0, max=5788)

Epoch 1/20
368/368 [==============================] - 8s 7ms/step - loss: 0.2381 - mean_absolute_error: 0.2946 - val_loss: 0.2511 - val_mean_absolute_error: 0.2682
Epoch 2/20
368/368 [==============================] - 2s 4ms/step - loss: 0.1986 - mean_absolute_error: 0.2717 - val_loss: 0.2530 - val_mean_absolute_error: 0.2707
Epoch 3/20
368/368 [==============================] - 2s 4ms/step - loss: 0.1975 - mean_absolute_error: 0.2713 - val_loss: 0.2440 - val_mean_absolute_error: 0.2720
Epoch 4/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1966 - mean_absolute_error: 0.2718 - val_loss: 0.2520 - val_mean_absolute_error: 0.2709
Epoch 5/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1972 - mean_absolute_error: 0.2722 - val_loss: 0.2451 - val_mean_absolute_error: 0.2652
***TRAIN***:
correlation related to cluster centre: 0.90
r2 score related to cluster centre: 0.81
***VALIDATION***:
correlation related to cluster centre: 0.89
r2 score related to 

IntProgress(value=0, max=5788)

***VALIDATION***:
average correlation: nan
average r2: -4115998449898177659486076928.00
---do_investigations. cluster index : 1---


...

***VALIDATION***:
average correlation: 0.91
average r2: 0.83
---do_investigations. cluster index : 10---


IntProgress(value=0, max=1)

Epoch 1/20
368/368 [==============================] - 18s 13ms/step - loss: 0.1944 - mean_absolute_error: 0.3032 - val_loss: 0.1817 - val_mean_absolute_error: 0.3040
Epoch 2/20
368/368 [==============================] - 2s 4ms/step - loss: 0.1678 - mean_absolute_error: 0.2818 - val_loss: 0.1806 - val_mean_absolute_error: 0.2918
Epoch 3/20
368/368 [==============================] - 1s 4ms/step - loss: 0.1665 - mean_absolute_error: 0.2799 - val_loss: 0.1810 - val_mean_absolute_error: 0.2958
Epoch 4/20
368/368 [==============================] - 1s 4ms/step - loss: 0.1666 - mean_absolute_error: 0.2801 - val_loss: 0.1806 - val_mean_absolute_error: 0.2997
***TRAIN***:
correlation related to cluster centre: 0.91
r2 score related to cluster centre: 0.84
***VALIDATION***:
correlation related to cluster centre: 0.91
r2 score related to cluster centre: 0.82


IntProgress(value=0, max=1)

***VALIDATION***:
average correlation: 0.91
average r2: 0.82
weighted_average_correlation: 0.90
weighted_average_r2_score: 0.76
bad clusters rate: 0.75%


In [31]:
first_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1.txt"
second_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 3 0.1 0.1.txt"
third_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 2 0.1 0.1.txt"
fourth_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 1 0.1 0.1.txt"

output_file = open("computational-experiments-output", "a")

selected_path = second_path

print(f"rnn {os.path.split(selected_path)[-1]}")
output_file.write(f"rnn {os.path.split(selected_path)[-1]}\n")
show_results(rnn_model, get_clusters(selected_path), CONV_WIDTH)

output_file.close()

rnn kohonen 15 0.1 0.1 - 3 0.1 0.1.txt
---do_investigations. cluster index : 0---


IntProgress(value=0, max=654)

Epoch 1/20
368/368 [==============================] - 13s 14ms/step - loss: 0.2269 - mean_absolute_error: 0.3361 - val_loss: 0.1936 - val_mean_absolute_error: 0.3029
Epoch 2/20
368/368 [==============================] - 1s 4ms/step - loss: 0.2032 - mean_absolute_error: 0.3162 - val_loss: 0.1923 - val_mean_absolute_error: 0.3042
Epoch 3/20
368/368 [==============================] - 1s 4ms/step - loss: 0.2032 - mean_absolute_error: 0.3161 - val_loss: 0.1960 - val_mean_absolute_error: 0.3018
Epoch 4/20
368/368 [==============================] - 1s 4ms/step - loss: 0.2028 - mean_absolute_error: 0.3164 - val_loss: 0.1923 - val_mean_absolute_error: 0.3000
Epoch 5/20
368/368 [==============================] - 1s 4ms/step - loss: 0.2030 - mean_absolute_error: 0.3164 - val_loss: 0.1928 - val_mean_absolute_error: 0.2986
Epoch 6/20
368/368 [==============================] - 1s 4ms/step - loss: 0.2038 - mean_absolute_error: 0.3173 - val_loss: 0.1910 - val_mean_absolute_error: 0.2999
Epoch 7/20
368

IntProgress(value=0, max=654)

***VALIDATION***:
average correlation: 0.90
average r2: 0.73
---do_investigations. cluster index : 1---


...

***VALIDATION***:
average correlation: 0.90
average r2: 0.79
---do_investigations. cluster index : 15---


IntProgress(value=0, max=5)

Epoch 1/20
368/368 [==============================] - 4s 5ms/step - loss: 0.2384 - mean_absolute_error: 0.2217 - val_loss: 0.3263 - val_mean_absolute_error: 0.2788
Epoch 2/20
368/368 [==============================] - 2s 4ms/step - loss: 0.1942 - mean_absolute_error: 0.2012 - val_loss: 0.3188 - val_mean_absolute_error: 0.2662
Epoch 3/20
368/368 [==============================] - 2s 4ms/step - loss: 0.1977 - mean_absolute_error: 0.2033 - val_loss: 0.3169 - val_mean_absolute_error: 0.2624
Epoch 4/20
368/368 [==============================] - 2s 4ms/step - loss: 0.1904 - mean_absolute_error: 0.1985 - val_loss: 0.3175 - val_mean_absolute_error: 0.2654
Epoch 5/20
368/368 [==============================] - 2s 4ms/step - loss: 0.1939 - mean_absolute_error: 0.2009 - val_loss: 0.3150 - val_mean_absolute_error: 0.2727
Epoch 6/20
368/368 [==============================] - 2s 4ms/step - loss: 0.1899 - mean_absolute_error: 0.1969 - val_loss: 0.3147 - val_mean_absolute_error: 0.2725
Epoch 7/20
368/3

IntProgress(value=0, max=5)

***VALIDATION***:
average correlation: 0.91
average r2: 0.82
weighted_average_correlation: 0.90
weighted_average_r2_score: 0.76
bad clusters rate: 0.59%


In [32]:
first_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1.txt"
second_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 3 0.1 0.1.txt"
third_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 2 0.1 0.1.txt"
fourth_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 1 0.1 0.1.txt"

output_file = open("computational-experiments-output", "a")

selected_path = third_path

print(f"rnn {os.path.split(selected_path)[-1]}")
output_file.write(f"rnn {os.path.split(selected_path)[-1]}\n")
show_results(rnn_model, get_clusters(selected_path), CONV_WIDTH)

output_file.close()

rnn kohonen 15 0.1 0.1 - 2 0.1 0.1.txt
---do_investigations. cluster index : 0---


IntProgress(value=0, max=222)

Epoch 1/20
368/368 [==============================] - 6s 6ms/step - loss: 0.2424 - mean_absolute_error: 0.3215 - val_loss: 0.1802 - val_mean_absolute_error: 0.2866
Epoch 2/20
368/368 [==============================] - 2s 4ms/step - loss: 0.2103 - mean_absolute_error: 0.2981 - val_loss: 0.1764 - val_mean_absolute_error: 0.2877
Epoch 3/20
368/368 [==============================] - 2s 4ms/step - loss: 0.2103 - mean_absolute_error: 0.2989 - val_loss: 0.1783 - val_mean_absolute_error: 0.2908
Epoch 4/20
368/368 [==============================] - 2s 4ms/step - loss: 0.2080 - mean_absolute_error: 0.2964 - val_loss: 0.1754 - val_mean_absolute_error: 0.2807
Epoch 5/20
368/368 [==============================] - 2s 4ms/step - loss: 0.2079 - mean_absolute_error: 0.2957 - val_loss: 0.1777 - val_mean_absolute_error: 0.2891
Epoch 6/20
368/368 [==============================] - 2s 4ms/step - loss: 0.2072 - mean_absolute_error: 0.2956 - val_loss: 0.1785 - val_mean_absolute_error: 0.2890
***TRAIN***:
cor

IntProgress(value=0, max=222)

***VALIDATION***:
average correlation: 0.90
average r2: 0.80
---do_investigations. cluster index : 1---


...

***VALIDATION***:
average correlation: 0.89
average r2: 0.80
---do_investigations. cluster index : 20---


IntProgress(value=0, max=164)

Epoch 1/20
368/368 [==============================] - 4s 6ms/step - loss: 0.2048 - mean_absolute_error: 0.2888 - val_loss: 0.1417 - val_mean_absolute_error: 0.2319
Epoch 2/20
368/368 [==============================] - 2s 4ms/step - loss: 0.1790 - mean_absolute_error: 0.2701 - val_loss: 0.1434 - val_mean_absolute_error: 0.2306
Epoch 3/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1800 - mean_absolute_error: 0.2712 - val_loss: 0.1411 - val_mean_absolute_error: 0.2300
Epoch 4/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1809 - mean_absolute_error: 0.2718 - val_loss: 0.1420 - val_mean_absolute_error: 0.2295
Epoch 5/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1788 - mean_absolute_error: 0.2685 - val_loss: 0.1420 - val_mean_absolute_error: 0.2336
***TRAIN***:
correlation related to cluster centre: 0.91
r2 score related to cluster centre: 0.82
***VALIDATION***:
correlation related to cluster centre: 0.92
r2 score related to 

IntProgress(value=0, max=164)

***VALIDATION***:
average correlation: 0.90
average r2: 0.75
weighted_average_correlation: 0.89
weighted_average_r2_score: 0.77
bad clusters rate: 0.53%


In [33]:
first_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1.txt"
second_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 3 0.1 0.1.txt"
third_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 2 0.1 0.1.txt"
fourth_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kohonen 15 0.1 0.1 - 1 0.1 0.1.txt"

output_file = open("computational-experiments-output", "a")

selected_path = fourth_path

print(f"rnn {os.path.split(selected_path)[-1]}")
output_file.write(f"rnn {os.path.split(selected_path)[-1]}\n")
show_results(rnn_model, get_clusters(selected_path), CONV_WIDTH)

output_file.close()

rnn kohonen 15 0.1 0.1 - 1 0.1 0.1.txt
---do_investigations. cluster index : 0---


IntProgress(value=0, max=269)

Epoch 1/20
368/368 [==============================] - 4s 5ms/step - loss: 0.2216 - mean_absolute_error: 0.3289 - val_loss: 0.1659 - val_mean_absolute_error: 0.2826
Epoch 2/20
368/368 [==============================] - 2s 4ms/step - loss: 0.1905 - mean_absolute_error: 0.3024 - val_loss: 0.1685 - val_mean_absolute_error: 0.2862
Epoch 3/20
368/368 [==============================] - 2s 4ms/step - loss: 0.1899 - mean_absolute_error: 0.3007 - val_loss: 0.1645 - val_mean_absolute_error: 0.2785
Epoch 4/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1910 - mean_absolute_error: 0.3015 - val_loss: 0.1639 - val_mean_absolute_error: 0.2784
Epoch 5/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1894 - mean_absolute_error: 0.3003 - val_loss: 0.1623 - val_mean_absolute_error: 0.2711
Epoch 6/20
368/368 [==============================] - 2s 5ms/step - loss: 0.1886 - mean_absolute_error: 0.2993 - val_loss: 0.1624 - val_mean_absolute_error: 0.2713
Epoch 7/20
368/3

IntProgress(value=0, max=269)

***VALIDATION***:
average correlation: 0.91
average r2: 0.77
---do_investigations. cluster index : 1---


...

***VALIDATION***:
average correlation: 0.89
average r2: 0.79
---do_investigations. cluster index : 44---


IntProgress(value=0, max=1)

Epoch 1/20
368/368 [==============================] - 4s 5ms/step - loss: 0.2568 - mean_absolute_error: 0.2073 - val_loss: 0.3712 - val_mean_absolute_error: 0.2392
Epoch 2/20
368/368 [==============================] - 1s 4ms/step - loss: 0.1980 - mean_absolute_error: 0.1825 - val_loss: 0.3498 - val_mean_absolute_error: 0.2379
Epoch 3/20
368/368 [==============================] - 1s 4ms/step - loss: 0.1911 - mean_absolute_error: 0.1797 - val_loss: 0.3658 - val_mean_absolute_error: 0.2276
Epoch 4/20
368/368 [==============================] - 1s 4ms/step - loss: 0.1909 - mean_absolute_error: 0.1819 - val_loss: 0.3469 - val_mean_absolute_error: 0.2483
Epoch 5/20
368/368 [==============================] - 1s 4ms/step - loss: 0.1918 - mean_absolute_error: 0.1820 - val_loss: 0.3519 - val_mean_absolute_error: 0.2275
Epoch 6/20
368/368 [==============================] - 1s 4ms/step - loss: 0.1915 - mean_absolute_error: 0.1814 - val_loss: 0.3468 - val_mean_absolute_error: 0.2337
Epoch 7/20
368/3

IntProgress(value=0, max=1)

***VALIDATION***:
average correlation: 0.90
average r2: 0.81
weighted_average_correlation: 0.89
weighted_average_r2_score: 0.76
bad clusters rate: 0.49%
